In [1]:
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten, Input
import csv
from numpy import array
from operator import itemgetter
import cv2
import os
import time
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import numpy as np
import tensorflow as tf

In [2]:
def create_labels(path):
    target = []
    target_a = []
    target_b = []
    target_c = []
    target_d = []
    target_e = []

    counter = {'MEL': 0, 'NV': 0, 'BCC': 0, 'AKIEC': 0, 'BKL': 0, 'DF': 0, 'VASC': 0}
    i = 0
    with open(path, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            if i == 0:
                i += 1
                continue
            if row[1] == '1.0':  # MEL
                counter['MEL'] += 1
                target.append(0)
                target_a.append(0)
                target_b.append(0)
                target_c.append(-1)
                target_d.append(-1)
                target_e.append(-1)

            elif row[2] == '1.0':  # NV
                counter['NV'] += 1
                target.append(1)
                target_a.append(0)
                target_b.append(1)
                target_c.append(-1)
                target_d.append(-1)
                target_e.append(-1)

            elif row[3] == '1.0':  # BCC
                counter['BCC'] += 1
                target.append(2)
                target_a.append(1)
                target_b.append(-1)
                target_c.append(0)
                target_d.append(-1)
                target_e.append(0)

            elif row[4] == '1.0':  # AKIEC
                counter['AKIEC'] += 1
                target.append(3)
                target_a.append(1)
                target_b.append(-1)
                target_c.append(0)
                target_d.append(-1)
                target_e.append(1)

            elif row[5] == '1.0':  # BKL
                counter['BKL'] += 1
                target.append(4)
                target_a.append(1)
                target_b.append(-1)
                target_c.append(1)
                target_d.append(0)
                target_e.append(-1)

            elif row[6] == '1.0':  # DF
                counter['DF'] += 1
                target.append(5)
                target_a.append(1)
                target_b.append(-1)
                target_c.append(1)
                target_d.append(1)
                target_e.append(-1)

            elif row[7] == '1.0':  # VASC
                counter['VASC'] += 1
                target.append(6)  # BCC
                target_a.append(1)
                target_b.append(-1)
                target_c.append(1)
                target_d.append(2)
                target_e.append(-1)

            else:
                continue
    print(counter)
    file.close()
    return target, target_a, target_b, target_c, target_d, target_e, counter

In [3]:
# imports all the images from a specified folder, with a specific extension
# and resizes to a specific imgHeight, imgWidth

def import_dataset(path, mode, fileExtension='.jpg', imgWidth=224, imgHeight=224):
    datasetFilenamesImages = []
    dataset = []
    print("Start importing " + mode + " images...")
    for filename in os.listdir(path):
        if filename.endswith(fileExtension): 
            completePath = os.path.join(path, filename)
            image = cv2.imread(completePath, cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, dsize=(imgHeight, imgWidth), interpolation=cv2.INTER_AREA)
            filenameImage = [filename, image]
            datasetFilenamesImages.append(filenameImage)
        else:
            continue
    
    datasetFilenamesImages = sorted(datasetFilenamesImages, key=itemgetter(0))
    for x in datasetFilenamesImages:
        dataset.append(x[1])
    
    return array(dataset)

In [4]:
def create_model(imgHeight = 224, imgWidth = 224):

    inputs = tf.keras.Input(shape=(imgHeight, imgWidth, 3)) # inputs.shape, inputs.dtype
    densenet = DenseNet121(include_top=False, weights='imagenet')
    x = densenet(inputs)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)

    a = Dense(2, activation='softmax', name="model_a")(x)
    b = Dense(2, activation='softmax', name="model_b")(x)
    c = Dense(2, activation='softmax', name="model_c")(x)
    d = Dense(3, activation='softmax', name="model_d")(x)
    e = Dense(2, activation='softmax', name="model_e")(x)

    model = tf.keras.Model(inputs=inputs, outputs=[a, b, c, d, e], name='global_loss')
    model.summary()
    return model

In [5]:
def to_categorical_custom(array, num_classes=2):
    categorical = []
    for i in range(len(array)):
        if array[i] == -1:
            categorical.append(np.zeros(num_classes, dtype=np.float))
        elif array[i] == 0:
            a = np.zeros(num_classes, dtype=np.float)
            a[0] = float(1)
            categorical.append(a)
        elif array[i] == 1:
            a = np.zeros(num_classes, dtype=np.float)
            a[1] = float(1)
            categorical.append(a)
        elif array[i] == 2:
            a = np.zeros(num_classes, dtype=np.float)
            a[2] = float(1)
            categorical.append(a)
    return categorical


In [6]:
def count_labels(array):
    d = {-1: 0, 0: 0, 1: 0, 2: 0}
    for i in range(len(array)):
        d[array[i]] += 1

    print(d)

In [ ]:
def scheduler(epoch):
    global no_epochs
    global lr
    if epoch < int(0.5 * no_epochs):
        return lr
    elif epoch < int(0.75 * no_epochs):
        return lr / 10.
    else:
        return lr / 100.

In [7]:
no_epochs = 25
lr = 1e-5

p_train = '/home/ruben/Desktop/isic_2018/train_2018'
t_train = '/home/ruben/Desktop/isic_2018/train_2018/labels.csv'

p_val = '/home/ruben/Desktop/isic_2018/val_2018'
t_val = '/home/ruben/Desktop/isic_2018/val_2018/labels.csv'

In [25]:
print(tf.executing_eagerly())
x_train = import_dataset(p_train, 'training')
x_val = import_dataset(p_val, 'val')

y_train, y_a, y_b, y_c, y_d, y_e, counter_train = create_labels(t_train)
y_val, y_val_a, y_val_b, y_val_c, y_val_d, y_val_e, counter_val = create_labels(t_val)

print("x_train size:", len(x_train))
print("y_train size:", len(y_train))

print("x_val size:", len(x_val))
print("y_val size:", len(y_val))

count_labels(y_a)
count_labels(y_b)
count_labels(y_c)
count_labels(y_d)
count_labels(y_e)


model = create_model()

losses = {
    "model_a": "categorical_crossentropy",
    "model_b": "categorical_crossentropy",
    "model_c": "categorical_crossentropy",
    "model_d": "categorical_crossentropy",
    "model_e": "categorical_crossentropy"
}

adam = Adam(lr=1e-5)
model.compile(optimizer=adam, loss=losses,
              metrics=["accuracy"])

y_a_cat = to_categorical_custom(y_a, num_classes=2) # fazer customizado,
y_b_cat = to_categorical_custom(y_b, num_classes=2)
y_c_cat = to_categorical_custom(y_c, num_classes=2)
y_d_cat = to_categorical_custom(y_d, num_classes=3)
y_e_cat = to_categorical_custom(y_e, num_classes=2)
y_val_a_cat = to_categorical_custom(y_val_a, num_classes=2)
y_val_b_cat = to_categorical_custom(y_val_b, num_classes=2)
y_val_c_cat = to_categorical_custom(y_val_c, num_classes=2)
y_val_d_cat = to_categorical_custom(y_val_d, num_classes=3)
y_val_e_cat = to_categorical_custom(y_val_e, num_classes=2)


y_val_cat = to_categorical(y_val)
tf.compat.v1.enable_eager_execution()
print(tf.executing_eagerly())

checkpoint_path = "/home/ruben/Desktop/isic_2018/model_weights/hier_shared_features/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1, save_best_only=True)

scheduler_cb = tf.keras.callbacks.LearningRateScheduler(scheduler)

class_weights_a = {0: 10608. / 8361, 1: 10608. / 2247}
class_weights_b = {0: 8361. / 1212, 1: 8361. / 7149}
class_weights_c = {0: 2247. / 879, 1: 2247. / 1368}
class_weights_d = {0: 1368. / 1101, 1: 1368. / 114, 2: 1368. / 153}
class_weights_e = {0: 879. / 519, 1: 879. / 360}
class_weights = [class_weights_a, class_weights_b, class_weights_c, class_weights_d, class_weights_e]
'''
print(len(x_train))
print(len(y_a_cat))
print(len(y_b_cat))
print(len(y_c_cat))
print(len(y_d_cat))
print(len(y_e_cat))

print(len(x_val))
print(len(y_val_a_cat))
print(len(y_val_b_cat))
print(len(y_val_c_cat))
print(len(y_val_d_cat))
print(len(y_val_e_cat))
'''
# Need to Fix this. For some reason it is not supported anymore...
fit = model.fit(x_train, [y_a_cat, y_b_cat, y_c_cat, y_d_cat, y_e_cat], batch_size=10, epochs=no_epochs, callbacks=[cp_callback, scheduler_cb], validation_data=(x_val, [y_val_a_cat, y_val_b_cat, y_val_c_cat, y_val_d_cat, y_val_e_cat])) # class_weights?

True
Start importing training images...
Start importing val images...
{'MEL': 3, 'NV': 3, 'BCC': 3, 'AKIEC': 3, 'BKL': 3, 'DF': 3, 'VASC': 3}
{'MEL': 1, 'NV': 1, 'BCC': 1, 'AKIEC': 1, 'BKL': 1, 'DF': 1, 'VASC': 1}
x_train size: 21
y_train size: 21
x_val size: 7
y_val size: 7
{-1: 0, 0: 6, 1: 15, 2: 0}
{-1: 15, 0: 3, 1: 3, 2: 0}
{-1: 6, 0: 6, 1: 9, 2: 0}
{-1: 12, 0: 3, 1: 3, 2: 3}
{-1: 15, 0: 3, 1: 3, 2: 0}
Model: "global_loss"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_35 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
densenet121 (Model)             multiple             7037504     input_35[0][0]                   
__________________________________________________________________________

<ipython-input-25-dccd58851a3e>:99: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arrayTruth = np.array(list_of_groundtruths)
<ipython-input-25-dccd58851a3e>:106: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arrayValTruth = np.array(list_of_val_groundtruths)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).